In [1]:
import scala.util.parsing.json._
import java.io.File
import java.io.PrintWriter

Intitializing Scala interpreter ...

Spark Web UI available at http://c100.local:8088/proxy/application_1596895008206_25801
SparkContext available as 'sc' (version = 2.4.0-cdh6.3.2, master = yarn, app id = application_1596895008206_25801)
SparkSession available as 'spark'


import scala.util.parsing.json._
import java.io.File
import java.io.PrintWriter


# Loading and Extracting Data in RDD

First, we load the data from HDFS into a RDD.

In [2]:
val DEVSET = "hdfs:///user/pknees/amazon-reviews/full/reviews_devset.json"
val FULLSET = "hdfs:///user/pknees/amazon-reviews/full/reviewscombined.json"
val reviewsFile = sc.textFile(FULLSET)

DEVSET: String = hdfs:///user/pknees/amazon-reviews/full/reviews_devset.json
FULLSET: String = hdfs:///user/pknees/amazon-reviews/full/reviewscombined.json
reviewsFile: org.apache.spark.rdd.RDD[String] = hdfs:///user/pknees/amazon-reviews/full/reviewscombined.json MapPartitionsRDD[1] at textFile at <console>:32


We now parse each line to json and extract the relevant data (category and review text) and create a paired RDD (key -> category, value -> reviewText) from it.

In [3]:
val pairedCategoryReview = reviewsFile.map(line => JSON.parseFull(line).get.asInstanceOf[Map[String, String]])
                                      .map(json => (json.get("category").get, json.get("reviewText").get))

pairedCategoryReview: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[3] at map at <console>:32


# Tokenization and Preprocessing

As we now store the full review text as String we have to perform Tokenization and some Preprocessing stpes -> Case Folding, StopWords removal, removing duplicates and removing tokens with only one character.

In [4]:
def tokenizePreprocess(reviewText: String): Array[String] = {
    val stopWords: Set[String] = Set("a", "aa", "able", "about", "above", "according", "accordingly", "across", "actually", "after", "afterwards", "again", "against", "ain", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "an", "and", "another", "any", "anybody", "anyhow", "anyone", "anything", "anyway", "anyways", "anywhere", "apart", "appear", "appreciate", "appropriate", "are", "aren", "around", "as", "aside", "ask", "asking", "associated", "at", "available", "away", "awfully", "b", "bb", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bibs", "book", "both", "brief", "but", "by", "c", "came", "can", "cannot", "cant", "car", "cause", "causes", "cd", "certain", "certainly", "changes", "clearly", "co", "com", "come", "comes", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "course", "currently", "d", "definitely", "described", "despite", "did", "didn", "different", "do", "does", "doesn", "doing", "don", "done", "down", "downwards", "during", "e", "each", "edu", "eg", "eight", "either", "else", "elsewhere", "enough", "entirely", "especially", "et", "etc", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "f", "far", "few", "fifth", "first", "five", "followed", "following", "follows", "for", "former", "formerly", "forth", "four", "from", "further", "furthermore", "g", "game", "game", "get", "gets", "getting", "given", "gives", "go", "goes", "going", "gone", "got", "gotten", "greetings", "h", "had", "hadn", "happens", "hardly", "has", "hasn", "have", "haven", "having", "he", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "hi", "him", "himself", "his", "hither", "hopefully", "how", "howbeit", "however", "i", "ie", "if", "ignored", "immediate", "in", "inasmuch", "inc", "indeed", "indicate", "indicated", "indicates", "inner", "insofar", "instead", "into", "inward", "is", "isn", "it", "its", "itself", "j", "just", "k", "keep", "keeps", "kept", "know", "known", "knows", "l", "last", "lately", "later", "latter", "latterly", "least", "less", "lest", "let", "life", "like", "liked", "likely", "little", "ll", "look", "looking", "looks", "ltd", "m", "mainly", "many", "may", "maybe", "me", "mean", "meanwhile", "merely", "might", "mon", "more", "moreover", "most", "mostly", "much", "must", "my", "myself", "n", "name", "namely", "nd", "near", "nearly", "necessary", "need", "needs", "neither", "never", "nevertheless", "new", "next", "nine", "no", "nobody", "non", "none", "noone", "nor", "normally", "not", "nothing", "novel", "now", "nowhere", "o", "obviously", "of", "off", "often", "oh", "ok", "okay", "old", "on", "once", "one", "ones", "only", "onto", "or", "other", "others", "otherwise", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "own", "p", "particular", "particularly", "per", "perhaps", "placed", "please", "plus", "possible", "presumably", "probably", "provides", "q", "que", "quite", "qv", "r", "rather", "rd", "re", "really", "reasonably", "regarding", "regardless", "regards", "relatively", "respectively", "right", "s", "said", "same", "saw", "say", "saying", "says", "second", "secondly", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "shall", "she", "should", "shouldn", "since", "six", "so", "some", "somebody", "somehow", "someone", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "specified", "specify", "specifying", "still", "sub", "such", "sup", "sure", "t", "take", "taken", "tell", "tends", "th", "than", "thank", "thanks", "thanx", "that", "that", "thats", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "there", "thereafter", "thereby", "therefore", "therein", "theres", "thereupon", "these", "they", "think", "third", "this", "thorough", "thoroughly", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "took", "toward", "towards", "tried", "tries", "truly", "try", "trying", "twice", "two", "u", "un", "under", "unfortunately", "unless", "unlikely", "until", "unto", "up", "upon", "us", "use", "used", "useful", "uses", "using", "usually", "v", "value", "various", "ve", "very", "via", "viz", "vs", "want", "wants", "was", "wasn", "way", "we", "welcome", "well", "went", "were", "weren", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "willing", "wish", "with", "within", "without", "won", "wonder", "would", "wouldn", "x", "y", "yes", "yet", "you", "your", "yours", "yourself", "yourselves", "z", "zero") 
    reviewText.toLowerCase.split("[ \t0123456789.!?,;:()\\[\\]{}\\-_\"'`~#&*%$\\\\/]+").toSet.diff(stopWords).toArray.filter(x => x.length() > 1)
}

tokenizePreprocess: (reviewText: String)Array[String]


In [5]:
val preprocessed = pairedCategoryReview.flatMapValues(x => tokenizePreprocess(x))

preprocessed: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[4] at flatMapValues at <console>:33


## Calculate A

To calculate A (number of combination of category and token) we crate a "composite key" consisting of (category, token) and as value we count the number of occurances.

In [6]:
val catTokenA = preprocessed.map(x => ((x._1, x._2), 1L)).reduceByKey((x, y) => x + y)

catTokenA: org.apache.spark.rdd.RDD[((String, String), Long)] = ShuffledRDD[6] at reduceByKey at <console>:31


## Calculate B

To calculate B (number of times token occures without the category) we remap the data to key -> token, value -> (category, A)

Then, we compute the number of times each token occures.

Next, we join the two RDDs by the key token and compute for each combination of category and token -> B = countToken - A

Last, we remap the data again to to the composite key (category, token)

In [7]:
// remap by token
val byToken = catTokenA.map(x => (x._1._2, (x._1._1, x._2)))

byToken: org.apache.spark.rdd.RDD[(String, (String, Long))] = MapPartitionsRDD[7] at map at <console>:32


In [8]:
// count total number each token occures -> countToken
val countByToken = byToken.reduceByKey((x, y) => ("N", x._2 + y._2)).map(x => (x._1, x._2._2))

countByToken: org.apache.spark.rdd.RDD[(String, Long)] = MapPartitionsRDD[9] at map at <console>:32


In [9]:
// join by key -> token and map to new value (category, A, countToken)
val tokenJoinedACount = byToken.join(countByToken).mapValues(x => (x._1._1, x._1._2, x._2))

tokenJoinedACount: org.apache.spark.rdd.RDD[(String, (String, Long, Long))] = MapPartitionsRDD[13] at mapValues at <console>:34


In [10]:
// calculate B = countToken - A and create new value (category, B)
val tokenJoinedCatB = tokenJoinedACount.mapValues(x => (x._1, x._3 - x._2))

tokenJoinedCatB: org.apache.spark.rdd.RDD[(String, (String, Long))] = MapPartitionsRDD[14] at mapValues at <console>:32


In [11]:
// remap to composite key -> (category, token) and value -> B
val catTokenB = tokenJoinedCatB.map(x => ((x._2._1, x._1), (x._2._2)))

catTokenB: org.apache.spark.rdd.RDD[((String, String), Long)] = MapPartitionsRDD[15] at map at <console>:32


## Calculate C

To calculate C (number of times category occures without the token) we remap the data to key -> category, value -> (token, A)

Then, we compute the number of times each category occures.

Next, we join the two RDDs by the key category and compute for each combination of category and token -> C = countCategory - A

Last, we remap the data again to to the composite key (category, token)

In [12]:
// remap by category
val byCat = catTokenA.map(x => (x._1._1, (x._1._2, x._2)))

byCat: org.apache.spark.rdd.RDD[(String, (String, Long))] = MapPartitionsRDD[16] at map at <console>:32


In [13]:
// count total number each category occures -> countCategory
val countByCat = byCat.reduceByKey((x, y) => ("N", x._2 + y._2)).map(x => (x._1, x._2._2))

countByCat: org.apache.spark.rdd.RDD[(String, Long)] = MapPartitionsRDD[18] at map at <console>:32


In [14]:
// join by key -> category and map to new value (token, A, countCategory)
val catJoinedACount = byCat.join(countByCat).mapValues(x => (x._1._1, x._1._2, x._2))

catJoinedACount: org.apache.spark.rdd.RDD[(String, (String, Long, Long))] = MapPartitionsRDD[22] at mapValues at <console>:34


In [15]:
// calculate C = countCategory - A and create new value (token, C)
val catJoinedTokenC = catJoinedACount.mapValues(x => (x._1, x._3 - x._2))

catJoinedTokenC: org.apache.spark.rdd.RDD[(String, (String, Long))] = MapPartitionsRDD[23] at mapValues at <console>:32


In [16]:
// remap to composite key -> (category, token) and value -> C
val catTokenC = catJoinedTokenC.map(x => ((x._1, x._2._1), (x._2._2)))

catTokenC: org.apache.spark.rdd.RDD[((String, String), Long)] = MapPartitionsRDD[24] at map at <console>:32


## Calculate N

To calculate N we calculate the sum of all counts by category

In [17]:
val N = countByCat.reduce((x, y) => ("N", x._2 + y._2))._2

N: Long = 2198531076


## Calculate D

To calculate D (the number of times not the category and not the token occures) we can join the calculated values of A, B and C over the composite key (category, token).

D = (N - (A + B + C))

In [18]:
// join over key (category, token), map to value (A, B, C)
val catTokenABC = catTokenA.join(catTokenB).join(catTokenC).mapValues(x => (x._1._1, x._1._2, x._2))

catTokenABC: org.apache.spark.rdd.RDD[((String, String), (Long, Long, Long))] = MapPartitionsRDD[31] at mapValues at <console>:36


In [19]:
// calculate D = (N - (A + B + C))
val catTokenABCD = catTokenABC.mapValues(x => (x._1, x._2, x._3, N - (x._1 + x._2 + x._3)))

catTokenABCD: org.apache.spark.rdd.RDD[((String, String), (Long, Long, Long, Long))] = MapPartitionsRDD[32] at mapValues at <console>:34


## Calculate Chi Squared

Now we have everything we need to calculate the chi squared values.

In [22]:
def calculateChiSquared(a: Long, b: Long, c: Long, d: Long, n: Long): Double = {
    var A = a.toDouble
    var B = b.toDouble
    var C = c.toDouble
    var D = d.toDouble
    var N = n.toDouble
    (N * math.pow((A * D) - (B * C), 2)) / ((A + B) * (A + C) * (B + D) * (C + D))
}

calculateChiSquared: (a: Long, b: Long, c: Long, d: Long, n: Long)Double


In [23]:
// calculate chi squared values by key -> (category, token)
val catTokenChiSquared = catTokenABCD.mapValues(x => calculateChiSquared(x._1, x._2, x._3, x._4, N))

catTokenChiSquared: org.apache.spark.rdd.RDD[((String, String), Double)] = MapPartitionsRDD[33] at mapValues at <console>:36


## Extract Top 150 Values

Now we have all chi squared values we can extract the top 150 values by each category.

For this we transform the data from key -> (category, token), value -> chi_squared to the new key value pair key -> category, value -> (token, chi_squared)

In [24]:
// map to key -> category, value -> (token, chi_squared)
val byCategoryChiSquared = catTokenChiSquared.map(x => (x._1._1, (x._1._2, x._2)))

byCategoryChiSquared: org.apache.spark.rdd.RDD[(String, (String, Double))] = MapPartitionsRDD[34] at map at <console>:32


In [25]:
// group by key -> category, sort chi_squared values descending and take top 150 token, chi_squared values by category
val top150ByCat = byCategoryChiSquared.groupByKey.mapValues(_.toList).mapValues(x => x.sortWith(_._2 > _._2).take(150))

top150ByCat: org.apache.spark.rdd.RDD[(String, List[(String, Double)])] = MapPartitionsRDD[37] at mapValues at <console>:32


# Extract distinct Tokens

Now we have the top 150 values by each category we can extract the distinct tokens.

In [26]:
val topTokens = top150ByCat.flatMap(x => x._2.map(a => a._1)).distinct().collect().sorted

topTokens: Array[String] = Array(ableton, absorbs, accessories, accord, acdelco, acne, acoustic, acre, acted, acting, action, actor, actors, actress, actresses, acura, adaptation, adapter, addario, addicted, addicting, addictive, adele, adf, adicting, adjustable, adorable, ads, adults, adventure, aftermarket, aftertaste, ages, air, aired, airprint, airsoft, akai, album, albums, alesis, almond, almonds, alpha, alternator, amazon, ameda, ammo, amp, amps, android, angry, animated, animation, anime, anker, ankle, answering, antenna, anthem, ants, apos, app, appetite, apple, appliance, apply, applying, apps, appstore, aquarium, aquila, ar, arcade, arch, argan, armband, aroma, arrangement, arrangements, arrived, arthritis, articulation, artificial, artist, artists, asics, asio, asleep, assemb...

# Output file

Last, we collect the top 150 token, chi_squared values by each category and output them in the desired format.

In [27]:
// collect data to map
val top150Map = top150ByCat.mapValues(x => x.map(a => a._1 + ":" + a._2)).collectAsMap()

top150Map: scala.collection.Map[String,List[String]] = Map(Electronic -> List(camera:3794149.488327162, laptop:3368202.5157042025, cable:3251535.347672754, usb:2668395.5417753896, computer:2566901.793393269, ipad:1826069.1926479763, screen:1714504.8668955013, speakers:1679762.9939352483, keyboard:1639875.5099000714, battery:1626385.0604032495, gb:1615016.2017846012, works:1594454.127553639, sound:1592926.3214825008, headphones:1579626.348952748, hdmi:1524298.2344598493, tv:1518372.4152058323, sony:1480804.7066151768, ipod:1469303.5251216711, device:1458835.2308369656, wireless:1418826.5103847797, adapter:1417616.0165455407, tablet:1384648.2477581592, drive:1271509.4414421022, plug:1255300.7967444498, card:1197511.9593801002, cables:1188544.14431392, pc:1162120.9435616126, software:11478...

In [28]:
// create output file and print writer
val myfile = new File("./output_rdd.txt" )
val pw = new PrintWriter(myfile)

myfile: java.io.File = ./output_rdd.txt
pw: java.io.PrintWriter = java.io.PrintWriter@10adaeca


In [29]:
// format data in desired format and wirte to file
for (cat <- top150Map.keySet.toSeq.sorted) {
    var output = cat
    top150Map.get(cat).get.foreach(x => output += " " + x)
    pw.write(output + "\n")
}

for (token <- topTokens) {
    pw.write(token + " ")
}

In [30]:
pw.close()